# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [ ]:
%pip install ipython-sql

In [80]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [81]:
# Query to retrieve tracks and their corresponding album names
query_tracks = """
SELECT tracks.name AS TrackName, albums.title AS AlbumName
FROM tracks
JOIN albums ON tracks.albumid = albums.albumid
LIMIT 5;
"""

# Read & store the result in a Pandas DataFrame
tracks = %sql $query_tracks

# Display the first 5 rows of the tracks data
display(tracks)


 * sqlite:///chinook.sqlite
Done.


TrackName,AlbumName
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Put The Finger On You,For Those About To Rock We Salute You
Let's Get It Up,For Those About To Rock We Salute You
Inject The Venom,For Those About To Rock We Salute You
Snowballed,For Those About To Rock We Salute You


In [82]:
#Query to find the total number of customers from each country 
query_customer = """
SELECT country, COUNT(customerId) AS TotalCustomers
FROM customers
GROUP BY country
ORDER BY TotalCustomers DESC;
"""
# Read & store the result in a Pandas DataFrame
customers = %sql $query_customer

display(customers)

 * sqlite:///chinook.sqlite
Done.


Country,TotalCustomers
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


In [83]:
#Identify the top 5 genres with the highest number of tracks. 
#Display the genre name along with the total number of tracks for each genre.
query_genres = """
SELECT genres.name AS Genre,
COUNT(tracks.trackId) AS TotalTracks
FROM genres
JOIN tracks ON genres.GenreID=tracks.GenreID
GROUP BY genres.Name
ORDER BY TotalTracks DESC
LIMIT 5
"""
genres = %sql $query_genres
display(genres)

 * sqlite:///chinook.sqlite
Done.


Genre,TotalTracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130


In [84]:
#Determine the average invoice total for each customer, considering both the album and individual track purchases. 
#Display the customer's first and last name along with the average invoice total.
#Order the results by the average invoice total in descending order.
query_customer = """
SELECT (c.firstname || ' ' || c.lastname) AS CustomerName,
AVG(i.Total) AS AverageInvoiceTotal
FROM customers c
JOIN invoices i ON c.customerId = i.customerId
GROUP BY c.customerId, c.firstname, c.lastname
ORDER BY AverageInvoiceTotal DESC;
"""
customer = %sql $query_customer
display(customer)

 * sqlite:///chinook.sqlite
Done.


CustomerName,AverageInvoiceTotal
Helena Holý,7.088571428571428
Richard Cunningham,6.8028571428571425
Luis Rojas,6.659999999999999
Ladislav Kovács,6.517142857142857
Hugh O'Reilly,6.517142857142857
Frank Ralston,6.231428571428571
Julia Barnett,6.231428571428571
Fynn Zimmermann,6.231428571428571
Puja Srivastava,6.1066666666666665
Astrid Gruber,6.088571428571428


In [85]:
#Identify the customer who spent the most on music purchases. 
#Display the customer's first and last name, along with the total amount spent.
query_purchases = """
SELECT c.firstname, c.lastname,
SUM(i.Total) AS TotalAmountSpent
FROM customers c
JOIN invoices i ON c.customerId=i.customerID
GROUP BY c.customerId, c.firstname, c.lastname
ORDER BY TotalAmountSpent DESC
LIMIT 1
"""
purchases = %sql $query_purchases
display(purchases)


 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,TotalAmountSpent
Helena,Holý,49.62
